In [112]:
import numpy
#import theano

Phase 1 - Scraping facebook posts.

The following code will scrape facebook posts from a public facebook group given the following parameters:
access_token - a facebook developers access token which is valid for 2 hours from the moment it is generated

you can get your own access token by following the instructions on: from https://developers.facebook.com/docs/facebook-login/access-tokens

**NOTE: THE ACCESS TOKEN HARDCODED HERE IS NO LONGER VALID BY THE TIME YOU READ THIS, SO IN ORDER TO RUN THE PROJECT YOU MUST CREATE YOUR OWN ACCESS TOKEN**

**NOTE: RUN THIS CODE WITH PYTHON 2 ONLY OR IT WON'T WORK**

This box of code creates a .csv file which contains the posts scraped from the given group.

In [ ]:
import urllib2
import json
import datetime
import csv
import time

#change the group ID to another group if you want to, currently it will scrape the group with the id 143136595007
group_id = "143136595007"

#THIS CODE IS FOR SCRAPING POSTS FROM A PUBLIC FACEBOOK GROUP GIVEN THE GROUP ID AND AN ACCESS TOKEN.
#THE ACCESS TOKEN IS TEMPORARY AND EXPIRES AFTER 2 HOURS.

#Enter your facebook developer's access token. Note: it will stop working after 2 hours, so make sure that after 2 hours you paste in 
#the new access token

access_token = "EAACEdEose0cBAO2UyNmch9icbTO9UpIV4gZBOBCYrCrPs7xBUlJO3CMNUJuZCDU5XT2HJmzYW8pKZCCKF8DRZCOFQjeiJZAEcLZCone7ZBQQmSV7Ia1TAtiKpvYZBivDIZBzsOCRZAN2i3eKrT6uKR3nSf8n0eBZC2uaJ5PjrjIzPlTPJUVt8I0qrmZBbvJFMTLZCfmAZD"


def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            print "Error for URL %s: %s" % (url, datetime.datetime.now())
            print "Retrying."

    return response.read()

# Needed to write tricky unicode correctly to csv
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22,
                            0xa0:0x20 }).encode('utf-8')

def getFacebookPageFeedData(group_id, access_token, num_statuses):
    # Construct the URL string; see 
    # http://stackoverflow.com/a/37239851 for Reactions parameters
    base = "https://graph.facebook.com/v2.6"
    node = "/%s/feed" % group_id 
    fields = "/?fields=message,link,permalink_url,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions." + \
            "limit(0).summary(true),from"
    parameters = "&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + fields + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))

    return data

def processFacebookPageFeedStatus(status, access_token):

    # We decided to make it so that the status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else \
            unicode_normalize(status['message'])
    link_name = '' if 'name' not in status.keys() else \
            unicode_normalize(status['name'])
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else \
            unicode_normalize(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_normalize(status['permalink_url'])
    status_author = unicode_normalize(status['from']['name'])
    status_published = datetime.datetime.strptime(\
            status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    # best time format for spreadsheet programs:
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S')

    return (status_id, status_message, status_author, link_name, status_type, 
            status_link, status_permalink_url, status_published)

def scrapeFacebookPageFeedStatus(group_id, access_token):
    with open('facebook_statuses.csv', 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "status_author", 
            "link_name", "status_type", "status_link","permalink_url",
            "status_published"])

        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()

        print "Scraping %s Facebook Page: %s\n" % \
                (group_id, scrape_starttime)

        statuses = getFacebookPageFeedData(group_id, access_token, 100)

        while has_next_page:
            for status in statuses['data']:
                if 'reactions' in status:            
                    w.writerow(processFacebookPageFeedStatus(status, \
                                  access_token))
                num_processed += 1
                if num_processed % 100 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, 
                            datetime.datetime.now())
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(\
                        statuses['paging']['next']))
            #dont scrape more than 6000 posts because it takes more than 2 hours and the access token will expire
            elif num_processed == 6000:
                has_next_page = False
            else:
                has_next_page = False

        print "\nDone!\n%s Statuses Processed in %s" % \
                (num_processed, datetime.datetime.now() - scrape_starttime)


if __name__ == '__main__':
    scrapeFacebookPageFeedStatus(group_id, access_token)


PHASE 1 (cont.) 
After we have the scraped posts in .csv files for the group/groups we want to subset the data and clear out unwanted posts. We also want to classify and separate male an female posts to 2 different .csv files: 1 for male posts and 1 for female posts. 

RUN THIS CODE WITH **PYTHON 3** ONLY OR IT WON'T WORK

Some notes regarding the data gathering part:
Facebook doesn't allow scraping people's personal statuses even if they are posted as 'public', but it does allow scraping
posts from GROUPS that are public. Unfortunately, this would mean that posts will not be more generic and probably correlate to the author's gender as the post is less likely to contain a 'personal tone' and focus more on the topic of the group. That means that the posts are less likely to truly represent male or female behavior because the posts will be more in the context of the group topic or a serious discussion. 

We decided to scrape posts from 2 different groups, both of which tech-oriented. The 1st group is called 'Big Data, Data Science, Data Mining & Statistics'
and the second group is called 'Science, Technology, and Society Discussion Corner'.
Naturally, that would mean the posts would likely focus on tech and data.

Another big problem is that Facebook doesn't allow us to have the gender of the post author, but will try to solve this problem later on.
See here: https://stackoverflow.com/questions/24945763/facebook-graph-api-how-to-get-gender-from-public-profile

In [255]:
import pandas as pd
#After scraping the posts from the 2 groups and inserting them into a .csv files, we read files and create a dataframe.
facebook_posts_group1 = pd.DataFrame.from_csv('143136595007_facebook_statuses.csv')
facebook_posts_group2 = pd.DataFrame.from_csv('484618291590939_facebook_statuses.csv')

Combine the 2 files into a single dataframe:

In [256]:
facebook_posts_selected_columns_group1 = facebook_posts_group1[['status_message','status_author', 'status_type']]
facebook_posts_selected_columns_group2 = facebook_posts_group2[['status_message','status_author', 'status_type']]

facebook_groups_posts = pd.concat([facebook_posts_selected_columns_group1, facebook_posts_selected_columns_group2], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)

facebook_groups_posts.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2240758702643547,NaN,Foued Saâdaoui,link
484618291590939_2239120939473990,If you want to solve some real-world problems ...,Matt Mayo,link
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238306732888744,NaN,Saimadhu Polamuri,link
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2237867916265959,NaN,Joseph Anoop George,status


Next:
CLEANING THE DATASET

remove statuses that are too short (less than 2 words) because they are more likely to be spam or meaningless.
Then remove rows containing these spam messages.

In [257]:
import numpy as np
posts_empties_removed = facebook_groups_posts.dropna(axis=0, how='any')

for index, row in posts_empties_removed.iterrows():
    if (len(row['status_message'].split(" ")) == 1):
        row['status_message'] = np.nan

posts_empties_removed2 = posts_empties_removed.dropna(axis=0, how='any')
posts_empties_removed2.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2239120939473990,If you want to solve some real-world problems ...,Matt Mayo,link
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2237858432933574,This post will take a different approach to co...,Matt Mayo,link
484618291590939_2237567572962660,#Reinforcementlearning to play an important ro...,Bharath Reddy,link
484618291590939_2236526149733469,"Riskified is hiring ! \nAs a Big Data expert, ...",Maya Salemnia,link


Throw away posts that are nothing but a link or a video description, because they do not contain text that was written by the author, it only contains the description of the link/video they posted.

In [258]:
posts_statuses_only = posts_empties_removed2[(posts_empties_removed2.status_type != 'link') & (posts_empties_removed2.status_type != 'video')]
posts_statuses_only.head(10)

,status_message,status_author,status_type
status_id,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status
484618291590939_2238261742893243,----------------------\nCall for Book Chapters...,Kuan-Ching Li,status
484618291590939_2236102949775789,Tweet Wars! Do you remember our sentiment anal...,Jamie Jones,photo
484618291590939_2232146760171408,Join the Big Data 5 Month Course + Project \nH...,Simran Kaur,photo
484618291590939_2230118397040911,If you are interested please drop me an email ...,Ritin Patel,photo
484618291590939_2224717017581049,"Most #banking, #financial services, and #insur...",Thomas Crystal,status


NEXT - DETERMINING THE GENDER OF USERS

Since Facebook won't allow us information such as the gender of the post author, we will have to turn to another solution. 
We found a python package that can guess the gender of a person judging by their first name using a dataset created from machine learning 
The package will determine if a name belongs to a male or female with 90+ accuracy. With 80+ accuracy, it will classify as mostly_male/mostly_female
If the accuracy is lower than 80%, it will classify as 'unknown'.
For the sake of accuracy, we will only keep those classified as male or female with 90+ accuracy.

First, we need to add a 'gender' column and a 'first_name' column, and then populate the gender column with the gender detector result.
Afterwards, we need to clear out those who were classified as 'mostly X' or 'unknown'.

In [259]:
 import gender_guesser.detector as gender

detector = gender.Detector()

lst = []
iterr = 0
posts_statuses_only['first_name'] = ""

#split the full names to first and last names
for index, row in posts_statuses_only.iterrows():
    lst.append(row['status_author'].split(" "))

#populate the first name coulmn with the first names from the splits
for index, row in posts_statuses_only.iterrows():
    row['first_name'] = lst[iterr][0]
    iterr = iterr+1

#populate the gender column with the class prediction using the gender guessing package
posts_statuses_only['gender'] = ""
iterr=0
for index, row in posts_statuses_only.iterrows():
    row['gender'] = detector.get_gender(row['first_name'])
    iterr = iterr+1


C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [260]:
#only the male gender part of the dataframe
male_posts = posts_statuses_only[posts_statuses_only["gender"] == "male"]
#only the female gender part of the dataframe
female_posts = posts_statuses_only[posts_statuses_only["gender"] == "female"]

In [261]:
#This is a dataframe on male posts
male_posts.head(10)

,status_message,status_author,status_type,first_name,gender
status_id,,,,,
484618291590939_2239065576146193,I am looking to learn R language.Please guide ...,Aditya Agarwal,status,Aditya,male
484618291590939_2238586102860807,Hi there\nDoes someone here have experience wo...,Julian Righ Sampedro,status,Julian,male
484618291590939_2238537122865705,"Help!! i need some subject, publication, book ...",Badr Rdab,status,Badr,male
484618291590939_2224717017581049,"Most #banking, #financial services, and #insur...",Thomas Crystal,status,Thomas,male
484618291590939_2218490481537036,Urgent Hiring for the following job SEO / SOCI...,Michael Christian Veras,photo,Michael,male
484618291590939_2207498585969559,"Hi, I'm searching a subject about combined int...",Badr Rdab,status,Badr,male
484618291590939_2203750419677709,I'm searching for case studies about Data Mini...,Carmelo Saffioti,status,Carmelo,male
484618291590939_2190444491008302,HAHAHA\nBitcoin Vs Money by Google Trend!\nBit...,Muhammad Fahad,status,Muhammad,male
484618291590939_2188021981250553,Walk-in for Data Engineering Professionals\n\n...,Mahesh Babu Doraivelu,event,Mahesh,male


In [262]:
#This is a dataframe of female posts
female_posts.head(10)

,status_message,status_author,status_type,first_name,gender
status_id,,,,,
484618291590939_2241938375858913,Oracle Code Bogotá y Buenos Aires !\n\nSi eres...,Juana Pinkalsky,status,Juana,female
484618291590939_2164214636964621,#BLACKFRIDAYDEAL This is the best offer I foun...,Martha Grace,photo,Martha,female
484618291590939_2149002608485824,"Hello Hyderabad,\nNow IIMSE Started BIGDATA HA...",Sandhya Chowdary,photo,Sandhya,female
484618291590939_2143841505668601,Hi All. Have anyone considered doing a researc...,Bala Sekar,status,Bala,female
484618291590939_2143552672364151,Don't forget to like our Facebook page 👍\n▶ ht...,Stella Morvan,photo,Stella,female
484618291590939_2143372895715462,Join our 48 hour hackathon on a moving train s...,Sarah Chok,event,Sarah,female
484618291590939_2137515939634491,Transform your organization with powerful anal...,Emily At Qlik,photo,Emily,female
484618291590939_2134485076604244,"Hello Guys,\n\nIf anybody is interested to pro...",Divya Tiwari,status,Divya,female
484618291590939_2114258738626878,Visit Spotinst at Serverlessconf NYC\nSpotinst...,Anat Kaplinski,photo,Anat,female


Calculate some basic statistics about out data:

In [263]:
#Statistics about our data:

#Let's calculate the males' average number of words in a post
posts_list_male = male_posts.status_message.tolist()

itemLength_males = 0
numOfItems_males = len(posts_list_male)
for item in posts_list_male:
    itemLength_males = itemLength_males + len(item.split())
    
avg_post_size_males = itemLength_males/numOfItems_males
print("Number of posts by males: ", numOfItems_males)
print("Average number of words per post for males: ", avg_post_size_males)


Number of posts by males:  561
Average number of words per post for males:  40.49910873440285


In [264]:
#Let's calculate the females' average number of words in a post
posts_list_females = female_posts.status_message.tolist()

itemLength_females = 0
numOfItems_females = len(posts_list_females)
for item in posts_list_females:
    itemLength_females = itemLength_females + len(item.split())
    
avg_post_size_females = itemLength_females/numOfItems_females
print("Number of posts by females: ", numOfItems_females)
print("Average number of words per post for females: ", avg_post_size_females)


Number of posts by females:  517
Average number of words per post for females:  36.49323017408124


In [265]:
#Extract the male DF to a .csv file:
male_posts.to_csv('male_posts.csv', sep=',', index=False)

#Extract the female DF to a .csv file:
female_posts.to_csv('female_posts.csv', sep=',', index=False)


Describing out data:

We scraped posts from 2 different facebook groups and extracted them to 2 .csv files - 1 for each group. Afterwards, we merged the 2 .csv files together to make a single dataset.
Then, we filtered out posts that were too short and posts that only contained a link or a video. facebook only allows us to see the posts content and the post author so had to predict the author's gender based on his/her first name using an external package. After predicting the user's gender, we filtered posts posted by users that we were uncertain about their gender based on the first name.
Finally, we separated male posts and female posts.
Eventually, we ended up with 2 .csv files:
male_posts - which contains 561 posts posted by males, with an average of 40 words per post.
female_posts - which contains 517 posts posted by females, with an average of 36 words per post.

These 2 datasets will be further analyzed when we use the next models.


**PHASE 2 - classification model (random forest)**

In [275]:
#Merge the 2 dataframes into one
data_female = pandas.read_csv('female_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data_male = pandas.read_csv('male_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data =  data_female.append(data_male, ignore_index=True)

#Now, we randomize the rows in order to eliminate any dependency in their current order
data_index =   numpy.random.permutation(len(data))
data2 = data
for i in range(len(data)):
    data2.iloc[i] = data.iloc[data_index[i]]
data = data2
data

,status_message,status_author,status_type,first_name,gender
0,For A Newbie What Is The Best Way To Start Wit...,Suma KP,photo,Suma,female
1,"Hi All,\r\n\r\nThis isn't a 'big' data questio...",Mike Gbny,status,Mike,male
2,"WOW! Northern Lights seen last night, Dec 11, ...",Rodrigo R. Menezes,status,Rodrigo,male
3,Anytime someone puts a lock on something you o...,George Cartwright,photo,George,male
4,"Can India Compete With China In #BigData, #Clo...",Suma KP,photo,Suma,female
5,"Despite undeniable progress, the public sector...",Jason Li,status,Jason,male
6,Dai Viet Group is recruting :\r\nJob title : D...,Rose Thuong,status,Rose,female
7,#Germany destroyed #Brazil by (7-1). Brazil ma...,Vineet Agarwal,photo,Vineet,male
8,"FOR JOB - ""Placement Co-Ordinators of all the ...",Vivek Shrivastava,status,Vivek,male
9,Preliminary Call for Papers; 6th International...,Marcin Paprzycki,status,Marcin,male


In [268]:
from bs4 import BeautifulSoup 
import re
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\rosen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\rosen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\rosen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\rosen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\rosen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [276]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [277]:
def posts_to_words( raw_post ):
    # 1. Remove HTML
    posts_text = BeautifulSoup(raw_posts, "html.parser").get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", posts_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [278]:
for i in range(len(data)):
    data["status_message"][i] = posts_to_words( data["status_message"][i] )
    print(data["status_message"][i])

newbie best way start bigdata technology join discussion visit http ow ly nhbhf hpc supercomputing
hi big data question going post think bad chance someone good suggestions data process process occurs low five figures per year originates dozen different settings variations process even merge want ignore process also handful different outcomes know duration process settings outcome types although one outcome un outcomed duration origin today rolling values long process take sometimes takes longer want one big face graphic shows table crude tree diagram need tree diagram template allows following origin show branches tree widths determined relative frequency source sub type process twigs coming branches also width determined relative frequency whole twigs vary length average duration sub type coming origin twig longer recommended time change green red anyone know preferably free software michael gibney
wow northern lights seen last night dec fairbanks alaska usa photo credit sacha layos 

remembering forrest j ackerman november december career literary agent ackerman represented science fiction authors ray bradbury isaac asimov e van vogt curtis siodmak l ron hubbard seven decades one science fiction staunchest spokesmen promoters ackerman editor principal writer magazine famous monsters filmland ackerman central formation organization spread science fiction fandom key figure wider cultural perception science fiction literary art film genre famous word play neologisms coined genre nickname sci fi
opportunities risks open syndicated data gartnerda opendata syndicateddata data datascience bigdata infopreneurship business
network models https www facebook com eurostatstatistics photos type
privacy preserving big data major issue please discuss regarding dat
anyone know access edx berkeleyx stat course showing enrollment closed
king tut wrap new technology coming wrap food produce include toxic waste dump longer needed surprisingly based old technology old one fact discover

want know transform text data valid data set make data mining please help step step
head dataframe df temperature dewpoint pressure date want select august august display temperature column trying df loc temperature throwing error generally speaking want learn using loc method easily take range row row k column j p show dataframe using loc method df loc k j p
j breaking news le patron airbnb rejoint les dirigeants du web b paris theevent h salle amphit atre keynote economie du partage uberisation qu allons nous devenir denis jacquet entrepreneur du net depuis fondateur de parrainer la croissance edufactory gr goire leclercq pr sident de la f ration des auto entrepreneurs vincent ricordeau co fondateur et pdg kisskissbankbank pascal picq pal oanthropologue tre de conf rences du coll ge de france olivier mathiot cofounder ceo priceminister sous reserve new nicolas ferrary ceo airbnb
anyone use boogie board lcd tablet
multiple openings engineering roles levels neogrowth credit pvt ltd int

god prankster gives instincts gives sets rules looks laughing ass set rules time tight ass sadist absentee landlord devil advocate https youtu irxqhgmqg
app workspace discovery demo http goo gl zsnbrm
herding unicorns manage data team british museum profusion datascience data business management
clouds cloud konomie http amzn wkctsn clouds economy http amzn uc yhx econom de las nubes http amzn tovc fanpage http bit ly thecloudseconomy
lonely one must first know
generalize free lunch theorem machine learning algorithms sense best ml method context independently
hey guys currently developing undergraduate thesis computer science looking problem related data science solve months would great problem also related sales marketing organizational development since like topics worked fields kind problems encountered trending applications data science right thanks response
tonic immobility ti
client technology startup online media space leveraging automation crowdsourcing create high quality med

aoa anyone tell applications factorial design
day workshop writing research proposal thesis university malaya kuala lumpur th april time pm speaker dr nasser alareqe international islamic university malaysia fee rm person
lad akins director special projects reef said one presentation long ago studies shown lionfish live without food months lose body mass https lionfish co lionfish faq
visual analytics ssrs ssas ios android windows https goo gl jwa w visual analytics ssas ssrs ios android windows course student experience analytics reporting visualizations would trained step step basics intention course empower students knowledge developing mobile dashboards senior leadership organization like ceo coo etc skills potentially yield salaries k based experience data visualizations mobile reporting techniques using microsoft bi tools using latest version sql server analysis services sql server reporting services course includes practical hands exercises well theoretical coverage key concepts

bhaskara supercomputer help meteorologist weather forecasting checkout news http goo gl fm dl hpc supercomputing
like painting elegance harmony
trending cont archives makes relevant imo goes
post poster engage software piracy deleted avoid imperilling group existence breaking fb rules instaban
going european conference data analysis yay ecda datascience statistics machinelearning dataanalysis essex bigdata
network models https www facebook com eurostatstatistics photos type
submit idea related transport bmvi earthobservation challenge digital transport applications exploit potential copernicus possible combination geo data develop new transport applications submission open july http bit ly otv
call position papers federated conference computer science information systems fedcsis submission deadline may http fedcsis org call position papers
bombard earth photons emit tesla roadster st
day workshop sem applications university malaya kuala lumpur th th september also accept local order pu

Create a countVectorizer to extract the features. We set maximum to 4000 features as it seemed to be enough and gave us good results.

In [353]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 4000) 


train_data_features = vectorizer.fit_transform(data["status_message"])

# Numpy arrays are easy to work with, so convert the result to an 
# array

train_data_features = train_data_features.toarray()

Creating the bag of words...



The training data array has 1527 rows and 4,000 features

In [354]:
print(train_data_features.shape)

(1527, 4000)


In [355]:
vocab = vectorizer.get_feature_names()
print(vocab[0:50])

['abap', 'aberdeenshire', 'abict', 'ability', 'able', 'aboard', 'abortion', 'absentee', 'absolute', 'absolutely', 'abstract', 'abstracts', 'ac', 'academic', 'academics', 'academy', 'accelerate', 'accelerator', 'accenture', 'accept', 'acceptance', 'accepted', 'accepting', 'access', 'accessible', 'accidental', 'according', 'accordingly', 'account', 'accountable', 'accounting', 'accuracy', 'accurate', 'ackerman', 'acm', 'acres', 'across', 'act', 'action', 'actions', 'activated', 'activities', 'actual', 'actually', 'actualmente', 'acuerdo', 'adapt', 'add', 'additional', 'address']


Now we split the data to test and train sets in order to train the model and test it's accuracy. This is very similar to an example shown in class.

In [356]:
from sklearn.ensemble import RandomForestClassifier
import numpy
#split to train & test
msk = numpy.random.rand(len(data)) < 0.8
train_x = train_data_features[msk]
test_x = train_data_features[~msk]
train_y = data.loc[msk,"gender"]
test_y = data.loc[~msk,"gender"]

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_x, train_y )

# Evaluate accuracy best on the test set
forest.score(test_x,test_y)

0.97819314641744548

We received an accuracy of 97%

Next, we need to generate new texts using keras (part 3) and run them on the trained model (part 4).

**PHASE 3 - GENERATING NEW TEXT WITH KERAS DEEP LEARNING PACKAGE**

In [113]:
#!pip install keras
import keras 
#print(keras.__version__)

First, we read the .csv files with the male and female posts. 

In [114]:
#!pip install pandas
import pandas 
data_female = pandas.read_csv('female_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data_male = pandas.read_csv('male_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data_female = data_female["status_message"]
data_male = data_male["status_message"]


Now we concatenate the female posts from a series to a single string of text. We start with the female posts, and in the next part we'll handle the male posts in the exact same manner.

In [115]:
female_text = ""
for i in range(len(data_female)):
    female_text = female_text + data_female[i] + " \n "  
female_text  

'Mohammed Nazili Suddicqui\'s post advertising a payment gateway is removed for a second time.  Third instance will result in the member being removed from the group. \n New member post advertising a payment gateway has been removed. \n This is amazing! \n We need 20 volunteers to go with us to Daytona Beach Florida for the NASCAR fundraiser! All expense paid! \n All presented by Alabama STEM Education! It will definitely move you to greatness? \n We may be older than we thought. \n Geostorm is a real thing on Mars. How well is SpaceX prepared for this problem? \n I need to interview two helping service professionals from two different settings (i.e. school, hospital, or prison), one of which must be a clinical psychologist, by this weekend.\r\nAnyone here fitting the description who could help? \n Any flat earthers in this group who\'d love a discussion?? \n Forgive me I couldn\'t resist. \n Amazing Mirrorless Car Camera Announced by Mitsubishi:\r\nhttp://www.techhound.org/auto-tech/a

In [116]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
import numpy


In [117]:
len(female_text)

141803

In [120]:
#tensorflow is the backend
from keras import backend as K
K.set_image_dim_ordering('tf')
%matplotlib inline
from keras.utils.np_utils import to_categorical

Now we clean the text - we remove unwanted characters and symbols. We will only leave characters that indicate the ending of a sentence or a new row, such as '\n'

In [121]:
female_text = female_text.replace(r'\[.*?\]|\(.*http.+\)|\(.*https.+\)|\<.*http.+\>', '')
female_text = female_text.replace(r'Rado([^\s]+)|Skarp([^\s]+)', '')
female_text = female_text.replace(r'\=[A-Z|0-9][A-Z|0-9]|\=', '')
female_text = female_text.replace('\r','')
female_text = female_text.replace('/r','')
female_text = female_text.replace('--',' ')
female_text= female_text.replace(r'[\(\)\;\-\\]','')
female_text

'Mohammed Nazili Suddicqui\'s post advertising a payment gateway is removed for a second time.  Third instance will result in the member being removed from the group. \n New member post advertising a payment gateway has been removed. \n This is amazing! \n We need 20 volunteers to go with us to Daytona Beach Florida for the NASCAR fundraiser! All expense paid! \n All presented by Alabama STEM Education! It will definitely move you to greatness? \n We may be older than we thought. \n Geostorm is a real thing on Mars. How well is SpaceX prepared for this problem? \n I need to interview two helping service professionals from two different settings (i.e. school, hospital, or prison), one of which must be a clinical psychologist, by this weekend.\nAnyone here fitting the description who could help? \n Any flat earthers in this group who\'d love a discussion?? \n Forgive me I couldn\'t resist. \n Amazing Mirrorless Car Camera Announced by Mitsubishi:\nhttp://www.techhound.org/auto-tech/amazi

Apply lower() to bring all the text to lower case letters.

In [122]:
female_text=female_text.lower()

Now we count the number of unique characters, to provide as a parameter the model.

In [123]:
chars = sorted(list(set(female_text)))
vocab_size = len(chars)
print('total chars:', vocab_size)

total chars: 77


In [124]:
''.join(chars) #the last six are unreadable
len(chars)

77

Char to int and int to char convertion (as was shown in class)

In [125]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Now we transform the female posts' text to integers.

In [126]:
idx = [char_indices[c] for c in female_text]
idx[:20]

[51, 53, 46, 39, 51, 51, 43, 42, 2, 52, 39, 64, 47, 50, 47, 2, 57, 59, 42, 42]

In [127]:
''.join(indices_char[i] for i in idx[:70])

"mohammed nazili suddicqui's post advertising a payment gateway is remo"

In [143]:
vec_len = 11
data_for_input = []
data_for_outpt = []
for i in range(0, len(idx) - vec_len, 1):   
    data_for_input.append([idx[j] for j in range(i,i+vec_len)])
    data_for_outpt.append(idx[i+vec_len])
n_patterns = len(data_for_input)


In [139]:
data_for_input

[[51, 53, 46, 39, 51, 51, 43, 42, 2, 52, 39],
 [53, 46, 39, 51, 51, 43, 42, 2, 52, 39, 64],
 [46, 39, 51, 51, 43, 42, 2, 52, 39, 64, 47],
 [39, 51, 51, 43, 42, 2, 52, 39, 64, 47, 50],
 [51, 51, 43, 42, 2, 52, 39, 64, 47, 50, 47],
 [51, 43, 42, 2, 52, 39, 64, 47, 50, 47, 2],
 [43, 42, 2, 52, 39, 64, 47, 50, 47, 2, 57],
 [42, 2, 52, 39, 64, 47, 50, 47, 2, 57, 59],
 [2, 52, 39, 64, 47, 50, 47, 2, 57, 59, 42],
 [52, 39, 64, 47, 50, 47, 2, 57, 59, 42, 42],
 [39, 64, 47, 50, 47, 2, 57, 59, 42, 42, 47],
 [64, 47, 50, 47, 2, 57, 59, 42, 42, 47, 41],
 [47, 50, 47, 2, 57, 59, 42, 42, 47, 41, 55],
 [50, 47, 2, 57, 59, 42, 42, 47, 41, 55, 59],
 [47, 2, 57, 59, 42, 42, 47, 41, 55, 59, 47],
 [2, 57, 59, 42, 42, 47, 41, 55, 59, 47, 9],
 [57, 59, 42, 42, 47, 41, 55, 59, 47, 9, 57],
 [59, 42, 42, 47, 41, 55, 59, 47, 9, 57, 2],
 [42, 42, 47, 41, 55, 59, 47, 9, 57, 2, 54],
 [42, 47, 41, 55, 59, 47, 9, 57, 2, 54, 53],
 [47, 41, 55, 59, 47, 9, 57, 2, 54, 53, 57],
 [41, 55, 59, 47, 9, 57, 2, 54, 53, 57, 58]

Now we convert this data structure from a list of lists to a ndarray type, and also we add another dimension because the model expects a 3d array.
140125 plus 11 are the number of characters in the female posts. 11 is the sequence size that iterates by 1.

In [140]:
x = (numpy.reshape(data_for_input, (n_patterns, vec_len, 1)))/ float(vocab_size)
x.shape

#this is a trick people use to fit the structure to the model. we found it online.

(140125, 11, 1)

Now we create a binary matrix in which each row represents one of the 140125 characters, and the columns represent all possible characters (77). Each row will show 1 if it corresponds with the column and 0 where it's not. This is needed as a parameter for the model.

In [131]:
y = to_categorical(data_for_outpt)
y.shape

(140125, 77)

Now we set the model. Using LSTM with hidden layer=256 because that's the size of the ASCII table. 

In [133]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers import SimpleRNN
from keras.callbacks import ModelCheckpoint
import keras
import tensorflow as tf


model = Sequential()




model.add(LSTM(256, input_length=x.shape[1], input_dim=x.shape[2]))
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))


C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(11, 1))`


set the wanted metric to accuracy and compile the model

In [134]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

100 epochs should be enough. The more the better, obviously, but the computation time is very long and we're short on time.

In [142]:
model.fit(x, y, epochs=100, batch_size=32,verbose=1)

Epoch 1/100
140096/140125 [============================>.] - ETA: 0s - loss: 2.9568 - acc: 0.1862
Epoch 00001: loss improved from inf to 2.95679, saving model to weights-improvement-01-2.9568.hdf5
140125/140125 [==============================] - 99s 705us/step - loss: 2.9568 - acc: 0.1862
Epoch 2/100
140064/140125 [============================>.] - ETA: 0s - loss: 2.8309 - acc: 0.2214
Epoch 00002: loss improved from 2.95679 to 2.83089, saving model to weights-improvement-02-2.8309.hdf5
140125/140125 [==============================] - 95s 677us/step - loss: 2.8309 - acc: 0.2214
Epoch 3/100
140064/140125 [============================>.] - ETA: 0s - loss: 2.7252 - acc: 0.2552
Epoch 00003: loss improved from 2.83089 to 2.72520, saving model to weights-improvement-03-2.7252.hdf5
140125/140125 [==============================] - 100s 714us/step - loss: 2.7252 - acc: 0.2552
Epoch 4/100
140064/140125 [============================>.] - ETA: 0s - loss: 2.6286 - acc: 0.2800
Epoch 00004: loss impro

Epoch 29/100
140064/140125 [============================>.] - ETA: 0s - loss: 1.3724 - acc: 0.6177
Epoch 00029: loss improved from 1.39742 to 1.37240, saving model to weights-improvement-29-1.3724.hdf5
140125/140125 [==============================] - 97s 690us/step - loss: 1.3724 - acc: 0.6177
Epoch 30/100
140064/140125 [============================>.] - ETA: 0s - loss: 1.3529 - acc: 0.6228
Epoch 00030: loss improved from 1.37240 to 1.35296, saving model to weights-improvement-30-1.3530.hdf5
140125/140125 [==============================] - 98s 703us/step - loss: 1.3530 - acc: 0.6228
Epoch 31/100
140064/140125 [============================>.] - ETA: 0s - loss: 1.3299 - acc: 0.6291
Epoch 00031: loss improved from 1.35296 to 1.32984, saving model to weights-improvement-31-1.3298.hdf5
140125/140125 [==============================] - 97s 695us/step - loss: 1.3298 - acc: 0.6291
Epoch 32/100
140032/140125 [============================>.] - ETA: 0s - loss: 1.3091 - acc: 0.6346
Epoch 00032: los

Epoch 57/100
140064/140125 [============================>.] - ETA: 0s - loss: 0.9921 - acc: 0.7135
Epoch 00057: loss improved from 0.99647 to 0.99217, saving model to weights-improvement-57-0.9922.hdf5
140125/140125 [==============================] - 97s 691us/step - loss: 0.9922 - acc: 0.7135
Epoch 58/100
140064/140125 [============================>.] - ETA: 0s - loss: 0.9838 - acc: 0.7153
Epoch 00058: loss improved from 0.99217 to 0.98389, saving model to weights-improvement-58-0.9839.hdf5
140125/140125 [==============================] - 98s 697us/step - loss: 0.9839 - acc: 0.7153
Epoch 59/100
140032/140125 [============================>.] - ETA: 0s - loss: 0.9764 - acc: 0.7161- ETA: 1s - 
Epoch 00059: loss improved from 0.98389 to 0.97652, saving model to weights-improvement-59-0.9765.hdf5
140125/140125 [==============================] - 98s 698us/step - loss: 0.9765 - acc: 0.7161
Epoch 60/100
140064/140125 [============================>.] - ETA: 0s - loss: 0.9681 - acc: 0.7190
Epoc

140125/140125 [==============================] - 101s 723us/step - loss: 0.8488 - acc: 0.7503
Epoch 85/100
140096/140125 [============================>.] - ETA: 0s - loss: 0.8436 - acc: 0.7523
Epoch 00085: loss improved from 0.84882 to 0.84371, saving model to weights-improvement-85-0.8437.hdf5
140125/140125 [==============================] - 97s 689us/step - loss: 0.8437 - acc: 0.7523
Epoch 86/100
140032/140125 [============================>.] - ETA: 0s - loss: 0.8387 - acc: 0.7521
Epoch 00086: loss improved from 0.84371 to 0.83868, saving model to weights-improvement-86-0.8387.hdf5
140125/140125 [==============================] - 100s 713us/step - loss: 0.8387 - acc: 0.7521
Epoch 87/100
140032/140125 [============================>.] - ETA: 0s - loss: 0.8371 - acc: 0.7546
Epoch 00087: loss improved from 0.83868 to 0.83701, saving model to weights-improvement-87-0.8370.hdf5
140125/140125 [==============================] - 99s 705us/step - loss: 0.8370 - acc: 0.7546
Epoch 88/100
140064/

In [145]:
model.evaluate(x,y)

140125/140125 [==============================] - 22s 157us/step


[0.68813033320758088, 0.80101338090777507]

As can be seen, we got an accuracy of 0.68, which is not very high but still ample.

Next, generate and predict the text. Eventually we output it to a list and then a file with the generated posts. Later, we'll do the same with the male posts.

In [210]:
def make_prediction(num_mess,length,model,data_for_input,leters_num):
    lst1=[]
    for j in range(num_mess):          
        n = numpy.random.randint(0, len(data_for_input)-1)
       
        arr_input = data_for_input[n]
        message=''.join([indices_char[value] for value in arr_input])
        for i in range(length):
           
            arr_input1 = numpy.reshape(arr_input, (1, len(arr_input), 1))
            arr_input2 = arr_input1 / float(leters_num)
            prediction = model.predict(arr_input2, verbose=0)
            index = numpy.argmax(prediction)
            result = indices_char[index]
            seq_in = [indices_char[value] for value in arr_input]
            message=message+result
            arr_input = arr_input[1:len(arr_input)]
            arr_input.append(index)
           
          
        lst1.append([message])
    return lst1

In [211]:
lst_female_new_posts=make_prediction(172,150,model,data_for_input,77)

In [212]:
lst_female_new_posts

[['entation an -d dimt oadesrrow fs ptr enb t couvter axmle.fnmdgaeoarn sf   bm osrrans ardlener cor aoe morn ihkor yhu mnlprn lepe metulng gidd saaee fu sor uo iut'],
 [' working on hionnr fer ssett rathir thar coel ris foarsiri ennftnde   thlt to dati tr be brtp io #vishlaeia llgttmnign?areou..#    pair ifudmt teshe tascemr io d'],
 ['ps://www.facebook.com/goeutse\n\niid f lausels dl comoiete i saekr \n arolnb aateo pfth rt ce aumliiesi  yrest onsslatehna s snetntes :oa wririmi tevhu teu bnn oo e'],
 ['.ly/2ius2gc \n anuone smowadii fi i fomdsinsils garo wh semrtetrtnr th anrpsna yeam bt entpie iot sorlremy ald altldey woali amamytics sasueriosnl tolgedisete \nh '],
 ['tolerance  \n  weelsae oi bawa itoe homosa.cesgec-co tuesgmtt of e bohhenadhra,dpd\npsttrdg aooeetts throirndnn ha wa \n\naas iiem ree seddcoiwr weensrmo yhuui easai'],
 ['me but what you la hamaoeon poiie \nctgr ayp :iadibrrentnses conlyi \n dnool fisr ob aimena th toannari\nbn. \n1aa ma mh dndda antdtryi ro foiegegni

In [213]:
pd_generate = pandas.DataFrame(data=lst_female_new_posts)
pd_generate

,0
0,executive\neligibility criteria:\nobcoon d/n...
1,pus.register here for free : http://bit.ly/2ns...
2,//www.udemy.com/dluster-analysis-motivation th...
3,es \n nouveab dooiiet pcoegl vhar ooeinilg cot...
4,n management aid data padested fnchymic dr aas...
5,-for-free-/119 \n henlo u wiilw to tasi forei...
6,loud storagess o\n ditmgn boa iuac sobdaa wiru...
7,dicion - octu pe saeitiegel tpdtiay uftifnt oa...
8,"ekend, a festive weekiad wo levr to tev ax ## ..."
9,ly and dema lnarltr frvp lo a frmcali anfigttt...


In [214]:
pd_generate.to_csv('generated_female_posts.csv', sep=',', index=False)

Now we repeat the exact same process with male posts, requires no explanation because its just a copy-paste from the previous part in our code.

In [215]:
male_text = ""
for i in range(len(data_male)):
    male_text = male_text + data_male[i] + " \n "  
male_text  

'Does Gmail sell information is one\'s private emails. Judging by adverts in my Facebook news feed, I would say "Yes." But perhaps this isn\'t news for anyone and I have been under a rock for years. Clarification, please. Is private email private in name only? \n Hold the applause. The new rocket has two main purposes, neither of them connected with the development of science: to boost the obscene personal wealth of Musk, currently more than $21 billion, and to expand the military arsenal of American imperialism.\r\n\r\n\'Falcon Heavy launch marks new stage in the privatization\x97and perversion\x97of space exploration\'\r\nWorld Socialist Web Site\r\n\r\nhttp://www.wsws.org/en/articles/2018/02/09/falc-f09.html \n Fake or real followers? \n THIS is where science is needed. HERE ON EARTH! \n If you bombard the Earth with photons for a while, it will emit a Tesla Roadster-st \n Wake Up America, the Original, the Original .. America is "Fighting The Wrong War"!  We need a "War On Killer D

In [240]:
male_text = male_text.replace(r'\[.*?\]|\(.*http.+\)|\(.*https.+\)|\<.*http.+\>', '')
male_text = male_text.replace(r'Rado([^\s]+)|Skarp([^\s]+)', '')
male_text = male_text.replace(r'\=[A-Z|0-9][A-Z|0-9]|\=', '')
male_text = male_text.replace('\r','')
male_text = male_text.replace('/r','')
male_text = male_text.replace('--',' ')
male_text= male_text.replace(r'[\(\)\;\-\\]','')
male_text = male_text.replace('\xad','')
male_text = male_text.replace('þ','')
male_text = male_text.replace('·','')
male_text = male_text.replace('\x99','')
male_text = male_text.replace('£','')
male_text

'does gmail sell information is one\'s private emails. judging by adverts in my facebook news feed, i would say "yes." but perhaps this isn\'t news for anyone and i have been under a rock for years. clarification, please. is private email private in name only? \n hold the applause. the new rocket has two main purposes, neither of them connected with the development of science: to boost the obscene personal wealth of musk, currently more than $21 billion, and to expand the military arsenal of american imperialism.\n\n\'falcon heavy launch marks new stage in the privatization\x97and perversion\x97of space exploration\'\nworld socialist web site\n\nhttp://www.wsws.org/en/articles/2018/02/09/falc-f09.html \n fake or real followers? \n this is where science is needed. here on earth! \n if you bombard the earth with photons for a while, it will emit a tesla roadster-st \n wake up america, the original, the original .. america is "fighting the wrong war"!  we need a "war on killer diseases"!!

In [241]:
male_text=male_text.lower()

In [242]:
chars2 = sorted(list(set(male_text)))
vocab_size2 = len(chars2)
print('total chars:', vocab_size2)

total chars: 71


In [243]:
idx2 = [char_indices[c] for c in male_text]
idx2[:20]

[42, 53, 43, 57, 2, 45, 51, 39, 47, 50, 2, 57, 43, 50, 50, 2, 47, 52, 44, 53]

In [244]:
vec_len2 = 11
data_for_input2 = []
data_for_outpt2 = []
for i in range(0, len(idx2) - vec_len2, 1):   
    data_for_input2.append([idx2[j] for j in range(i,i+vec_len2)])
    data_for_outpt2.append(idx2[i+vec_len2])
n_patterns2 = len(data_for_input2)

In [245]:
x2 = (numpy.reshape(data_for_input2, (n_patterns2, vec_len2, 1)))/ float(vocab_size2)
x2.shape

(295358, 11, 1)

In [246]:
y2 = to_categorical(data_for_outpt2)
y2.shape

(295358, 73)

In [249]:
model2 = Sequential()




model2.add(LSTM(256, input_length=x2.shape[1], input_dim=x2.shape[2])) #256 is the ascii size, x.shape[1] == 11 (size of seq), x.shape[2] = 1
model2.add(Dense(y2.shape[1]))##y.shape[1] = 77
model2.add(Activation('softmax'))


C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\rosen\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, input_shape=(11, 1))`
  


In [250]:
model2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model2.fit(x2, y2, epochs=100, batch_size=32,verbose=1)

Epoch 1/100
295358/295358 [==============================] - 198s 670us/step - loss: 2.9986 - acc: 0.1846
Epoch 2/100
295358/295358 [==============================] - 207s 699us/step - loss: 2.8838 - acc: 0.2079
Epoch 3/100
295358/295358 [==============================] - 198s 671us/step - loss: 2.8311 - acc: 0.2207
Epoch 4/100
295358/295358 [==============================] - 208s 704us/step - loss: 2.7827 - acc: 0.2342
Epoch 5/100
295358/295358 [==============================] - 204s 691us/step - loss: 2.7284 - acc: 0.2505
Epoch 6/100
295358/295358 [==============================] - 206s 697us/step - loss: 2.6735 - acc: 0.2652
Epoch 7/100
295358/295358 [==============================] - 196s 663us/step - loss: 2.6223 - acc: 0.2783
Epoch 8/100
295358/295358 [==============================] - 202s 683us/step - loss: 2.5758 - acc: 0.2933
Epoch 9/100
295358/295358 [==============================] - 204s 691us/step - loss: 2.5357 - acc: 0.3055
Epoch 10/100
295358/295358 [==================

295358/295358 [==============================] - 200s 676us/step - loss: 1.6638 - acc: 0.5393
Epoch 78/100
295358/295358 [==============================] - 198s 670us/step - loss: 1.6602 - acc: 0.5393
Epoch 79/100
295358/295358 [==============================] - 198s 669us/step - loss: 1.6543 - acc: 0.5404
Epoch 80/100
295358/295358 [==============================] - 199s 675us/step - loss: 1.6528 - acc: 0.5410
Epoch 81/100
295358/295358 [==============================] - 202s 684us/step - loss: 1.6472 - acc: 0.5418
Epoch 82/100
295358/295358 [==============================] - 195s 659us/step - loss: 1.6433 - acc: 0.5438
Epoch 83/100
295358/295358 [==============================] - 194s 655us/step - loss: 1.6404 - acc: 0.5444
Epoch 84/100
295358/295358 [==============================] - 196s 663us/step - loss: 1.6355 - acc: 0.5451
Epoch 85/100
295358/295358 [==============================] - 195s 660us/step - loss: 1.6317 - acc: 0.5469
Epoch 86/100
295358/295358 [======================

In [251]:
lst_male_new_posts=make_prediction(303,200,model2,data_for_input2,76)

In [252]:
#write to file
pd_generate_males = pandas.DataFrame(data=lst_male_new_posts)
pd_generate_males


,0
0,es of computtims. and isoiet nn therrmegoe ilr...
1,st of them tn tt pook to thralive pnakcdlm ne ...
2,lative to aut intl ro aiyan \n i aanertig oy ...
3,"fidelity iosi nr a dad .h, \n\n soeeiennst,a..."
4,torney lay'b.me bne pncelream ior fobln. sones...
5,will dramate dnt orog tfcer?n.welhna/s\n\n1p n...
6,"search on aywoaatil waeonta ana a, \n ie youea..."
7,", how he/sh pu e9ce\n siehse dv dlroass? \not..."
8,ple:-\nfirste poteneme a catkh oote on are htm...
9,e. \n when rreeri aelitisna tw toins nodir fn ...


In [253]:
pd_generate_males.to_csv('generated_male_posts.csv', sep=',', index=False)

The generated posts we got are mostly gibberish. That's because the generating process was character-based, meaning the next character is generated based on the previous 11 characters and so the model doesn't really consider the word's meaning or context in a sentence. However, some parts of the generated words resemble some kind of familiar meaning.

**PART 4- run the generated texts on the trained model****

In this part we need to take the generated posts and run them on the model from part 2 using the model's predict function.

In [358]:
#Merge the 2 dataframes into one
data_gen_female = pandas.read_csv('generated_female_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data_gen_male = pandas.read_csv('generated_male_posts.csv', sep=',', na_values=".", encoding='ISO-8859-1')
data_gen =  data_gen_female.append(data_gen_male, ignore_index=True)

#Now, we randomize the rows in order to eliminate any dependency in their current order
data_index =   numpy.random.permutation(len(data_gen))
data_gen2 = data_gen
for i in range(len(data_gen)):
    data_gen2.iloc[i] = data_gen.iloc[data_index[i]]
data_gen = data_gen2
data_gen

,0
0,ta in financial deiarasalc luree/seo piielsert...
1,.com/stormhydcsorcue eu mogirayel tsot ifmlyit...
2,s? you can refer : http://bit.ly/2nsbtfa \r\n ...
3,mputing is de pedhkng ciarrt snoeni to thareyr...
4,":o \r\n forcesi ts b aa deide? ?,? ;5h? 1rtwmo..."
5,"ational loner aeairt,\r\n\r\nstiane sud iuo ii..."
6,ta testing. wuerd is me ier bse crglyst.2 an \...
7,"get lifetim 1 toleseriss, iet oogrri yotel roe..."
8,lative to aut intl ro aiyan \r\n i aanertig o...
9,"that i get sanpn mncke enasa ,( h wor bnsote ..."


In [359]:
for i in range(len(data_gen)):
    data_gen["0"][i] = review_to_words( data_gen["0"][i] )
    print(data_gen["0"][i])

ta financial deiarasalc luree seo piielsert dle ladeis di dapaa cr fato iosetyieis lesi aclue mewe doselc botdy ynhdsa ss foinalrrg h crcn
com stormhydcsorcue eu mogirayel tsot ifmlyiti tp thenry aod puteari ar aoe toplne traie oo e memre incec rn ol teku thrtle trte blldcnared mttp whe whs oewrerlra aedi trinote wraiking aa redontel
refer http bit ly nsbtfa cog ysuiinrte aomuebt motaotsvsn fva htte tton mo thlg oemuereoene torufnsta iopo uet oadancmmr aog doottar
mputing de pedhkng ciarrt snoeni thareyrin aad e se aa ooavnlynegle tr tto ow tanenaer tichld css oh yoe trenet eoglnst ennogtdted oo rutemam eotrgrranin tafei ir sertoo li toener ta tuie
forcesi ts b aa deide h rtwmo tntigrnatl therrmrg bnl tre rf toewer kfrpsi lata rtogcsls date oastsselis sdo tr ialyem mi teskrc toailnsm daa ieis ropulnley wamune
ational loner aeairt stiane sud iuo iiteats anyroana oa unoasree tale erap eatf iireiene thaecci menoon ble tu setpnc put tacg ar zanen nete ou eala hfadce ko iart rerene ia w ser

l univemrna ine toas oen oover rf snti iur porhs tn nh horoad ar fwnpm nnre inr hodlc edda mriu ald n navr sonin therimans ior fupoinins thia ieraia roa whe bld hotaamihtracd irt aagkpraldao
hich open ss tont ieoeyie les ane mo ioreill ianeope anf frmmnb wt eeroite oilhnn sinnsi ior adddcss dat tony iatamakd iouun estiyeted ro aaeadi tiddnu oooere mtaer nomdresyi hnrtseisg fr
ge nsareyenec de lt eetoooab e tlihrroolhescy pontt innrt oo irrloweeieni aie riopornat bo mwir petda bt hiet eagksl rhrtod ioo ard wpe lulat seiir po aleiyhh ar tnscn poderenas aele
litate exceea idls mi eidnnen te sfirsg tentra orre tt aoo ar sosnn pr eetas h ceediater trof meme dreltse eoa bn loneltgg ar amgrka oo tuttraiee iaea hg ttip ie ao ziymg htdl tlnis
line frepmnig hnoint ta toeomnsg hsrmoria bt aluins apd crtlimsateo
ational loner aeairt stiane sud iuo iiteats anyroana oa unoasree tale erap eatf iireiene thaecci menoon ble tu setpnc put tacg ar zanen nete ou eala hfadce ko iart rerene ia w serstsca se


In [360]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 4000) 

train_data_features = vectorizer.fit_transform(data_gen["0"])


train_data_features = train_data_features.toarray()

Creating the bag of words...



In [361]:
print(train_data_features.shape)

(475, 4000)


In [362]:
vocab = vectorizer.get_feature_names()
print(vocab[0:50])

['aa', 'aaa', 'aad', 'aada', 'aade', 'aadea', 'aadi', 'aadl', 'aadle', 'aaduelatim', 'aae', 'aaeadi', 'aaecobotk', 'aaeee', 'aaeepsad', 'aagastd', 'aagkpraldao', 'aagkpraldaoesisvo', 'aainc', 'aala', 'aalaoil', 'aalatsee', 'aalestoa', 'aalet', 'aaleta', 'aaletal', 'aaletsood', 'aam', 'aami', 'aandsc', 'aane', 'aanele', 'aanert', 'aanertig', 'aanetsa', 'aantondda', 'aanyy', 'aaoutr', 'aaritn', 'aas', 'aasteldteneek', 'aat', 'aataacoide', 'aateo', 'aateu', 'aatieesnnn', 'aatt', 'aavutnatdteeesaa', 'aaxaar', 'aayetv']


In [364]:
from sklearn.ensemble import RandomForestClassifier

for index, row in data_gen.iterrows():
    row["0"] = row.replace('#',' ')
    row["0"] = row.replace('?',' ')
    row["0"] = row.replace('|',' ')
    row["0"] = row.replace(',', ' ')


predictions = forest.predict(X=test_x)

In [367]:
df_prediction = pandas.DataFrame(data=predictions)

df_prediction["gen_status"] = data_gen["0"]

Eventually, the model predicted the following genders to the following statuses:
*the 0's were probably added by the model for some reason, they should be ignored.

In [371]:
df_prediction

,0,gen_status
0,male,0 0 0 0 0 0 0 0 ta fin...
1,female,0 0 0 0 0 0 0 0 com st...
2,male,0 0 0 0 0 0 0 0 refer ...
3,male,0 0 0 0 0 0 0 0 mputin...
4,female,0 0 0 0 0 0 0 0 forces...
5,female,0 0 0 0 0 0 0 0 ationa...
6,female,0 0 0 0 0 0 0 0 ta tes...
7,female,0 0 0 0 0 0 0 0 get li...
8,male,0 0 0 0 0 0 0 0 lative...
9,male,0 0 0 0 0 0 0 0 get sa...


In [377]:
print("total number of predicted posts: ")
len(df_prediction)

total number of predicted posts: 


321

In [378]:
print("total number generated posts predicted to be female: ")
len(df_prediction[df_prediction[0]=='female'])

total number generated posts predicted to be female: 


45

In [379]:
print("total number generated posts predicted to be male: ")
len(df_prediction[df_prediction[0]=='male'])

total number generated posts predicted to be male: 


276

Final notes and improvements:

We can see that the generated posts were mostly classified as male. that could be due to the fact that we scraped more male posts (1010) than female posts (571). It just so happens that more males commented in the public groups we scraped, and so they had a bigger impact in teaching the model. 
Improvement: As mentioned in the beginning of the assignment, we were only able to retrieve status messages, and the status author's name. We could NOT get the gender as facebook doesn't allow it. If we had a way to get the real gender of each author then the model would be a bit more accurate than to use an external classification package in order to determine the author's gender based on his/her name. 
Also, facebook did not allow scraping simply any group, but only public groups. So it was hard finding groups where the was an equal amount of activity of males and females, and that the statuses in those groups are not spam or commercialized material.
Also, using 100 epoch wasn't optimal. The more epochs, the better, but as mentioned we were short on time. 
In addition to that, it is quite possible that using a model that generates text based on characters was not the best way to go, and we should have made use of other variants of models that generate text based on entire words.